# This is test jupyter notebook file, it tests the "tools/get_precipitation_data.py".

## Note: Some of the functions in the "tools/get_precipitation_data.py" file operate under specific assumptions, and the expected output is unpredictable even under these assumptions. Therefore, they were not tested in this test, but they all working properly in the project jupyter notebook files(try to re-run all the jupyter notebooks under from source code folder)

In [1]:
# Prepare a bit of packages:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
tool_folder_dir = "/content/gdrive/MyDrive/irp_project_111/source code"
sys.path.append(tool_folder_dir)
!pip install ipytest

Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.7 MB/s eta 0:00:00


In [2]:
"""
Author: Chaofan Wu
Student ID: 02285924
Email: cw522@ic.ac.uk
Project Name: Predicting flood risk in Ghana
Supervisors:
    Sesinam Dagadu(MEng)
    Yves Plancherel(PhD)
Company: SnooCODE
Date: 08/2023
"""

import ipytest
ipytest.autoconfig()
from tools import get_precipitation_data
import ee
import pandas as pd
from io import StringIO
import numpy as np
import pickle


def test_flatten_list():
    assert get_precipitation_data.flatten_list([[1], [2], [3]]) == [1, 2, 3]
    assert get_precipitation_data.flatten_list([1, 2, 3]) == [1, 2, 3]
    assert get_precipitation_data.flatten_list([[1.5], [2.5], [3.5]]) == [1.5, 2.5, 3.5]

def test_mean_calculator():
    assert get_precipitation_data.mean_calculator([[1], [2], [3]]) == 2.0
    assert get_precipitation_data.mean_calculator([1, 2, 3]) == 2.0
    assert get_precipitation_data.mean_calculator([[1.5], [2.5], [3.5]]) == 2.5
    assert get_precipitation_data.mean_calculator([1.5, 2.5, 3.5]) == 2.5

def test_median_calculator():
    assert get_precipitation_data.median_calculator([[1], [2], [3], [4]]) == 2.5
    assert get_precipitation_data.median_calculator([1, 2, 3, 4]) == 2.5
    assert get_precipitation_data.median_calculator([[1.5], [2.5], [3.5], [4.5]]) == 3.0
    assert get_precipitation_data.median_calculator([1.5, 2.5, 3.5, 4.5]) == 3.0

def test_max_continuous_days_calculator():
    assert get_precipitation_data.max_continuous_days_calculator([[0], [2], [3], [0], [4], [5], [6]]) == 3
    assert get_precipitation_data.max_continuous_days_calculator([0, 2, 3, 0, 4, 5, 6]) == 3

def test_max_continuous_rainfall_calculator():
    assert get_precipitation_data.max_continuous_rainfall_calculator([[0], [2], [3], [0], [4], [5], [6]]) == 15
    assert get_precipitation_data.max_continuous_rainfall_calculator([0, 2, 3, 0, 4, 5, 6]) == 15

def test_max_single_day_rainfall_calculator():
    assert get_precipitation_data.max_single_day_rainfall_calculator([[0], [2], [3], [0], [4], [5], [6]]) == 6
    assert get_precipitation_data.max_single_day_rainfall_calculator([0, 2, 3, 0, 4, 5, 6]) == 6

def test_max_rainfall_increase_calculator():
    assert get_precipitation_data.max_rainfall_increase_calculator([[0], [2], [3], [0], [4], [5], [6]]) == 4
    assert get_precipitation_data.max_rainfall_increase_calculator([0, 2, 3, 0, 4, 5, 6]) == 4

# Replace the path with a real or mock path to your pickle file containing the daily precipitation data
pickle_file_path = "/content/gdrive/MyDrive/irp_project_111/data/Daily_Precipitation/daily_precip_Modeling.pickle"

def test_calculate_rainfall_features():
    # Create a sample DataFrame
    sample_df = pd.DataFrame({
        'Flood_ID': ['A_1', 'B_2'],
        'Lon': [0, 1],
        'Lat': [0, 1],
    })

    # Mock the data in the pickle file
    mock_data = {
        'A_1_0_0': [1, 2, 3],
        'B_2_1_1': [4, 5, 6],
    }

    with open(pickle_file_path, 'wb') as f:
        pickle.dump(mock_data, f)

    # Create a list of feature calculators and corresponding column names
    feature_calculators = [np.mean, np.sum]
    column_names = ['Avg_Rain', 'Total_Rain']

    # Run the function
    result_df = get_precipitation_data.calculate_rainfall_features(sample_df, pickle_file_path, feature_calculators, column_names)

    # Check if the new columns are added
    for col in column_names:
        assert col in result_df.columns

    # Check if the values in the new columns are correct
    assert result_df['Avg_Rain'].tolist() == [np.mean([1, 2, 3]), np.mean([4, 5, 6])]
    assert result_df['Total_Rain'].tolist() == [np.sum([1, 2, 3]), np.sum([4, 5, 6])]

    # Add more edge cases, like an empty DataFrame or invalid input, if needed

def test_add_mean_median_precip():
    df = pd.DataFrame({'Flood_ID': ['F1', 'F2'], 'Lon': [100, 101], 'Lat': [10, 11]})
    updated_df = get_precipitation_data.add_mean_median_precip(df, pickle_file_path)
    assert 'Mean_Rainfall' in updated_df.columns
    assert 'Median_Rainfall' in updated_df.columns

def test_add_max_continuous_days_precip():
    df = pd.DataFrame({'Flood_ID': ['F1', 'F2'], 'Lon': [100, 101], 'Lat': [10, 11]})
    updated_df = get_precipitation_data.add_max_continuous_days_precip(df, pickle_file_path)
    assert 'Max_Continuous_Rainy_Days' in updated_df.columns

def test_add_max_continuous_precip():
    df = pd.DataFrame({'Flood_ID': ['F1', 'F2'], 'Lon': [100, 101], 'Lat': [10, 11]})
    updated_df = get_precipitation_data.add_max_continuous_precip(df, pickle_file_path)
    assert 'Max_Continuous_Rainfall' in updated_df.columns

def test_add_max_single_day_precip():
    df = pd.DataFrame({'Flood_ID': ['F1', 'F2'], 'Lon': [100, 101], 'Lat': [10, 11]})
    updated_df = get_precipitation_data.add_max_single_day_precip(df, pickle_file_path)
    assert 'Max_Single_Day_Rainfall' in updated_df.columns

def test_add_max_precip_increase():
    df = pd.DataFrame({'Flood_ID': ['F1', 'F2'], 'Lon': [100, 101], 'Lat': [10, 11]})
    updated_df = get_precipitation_data.add_max_precip_increase(df, pickle_file_path)
    assert 'Max_Rainfall_Increase' in updated_df.columns

def test_combine_precip():
    df = pd.DataFrame({'Flood_ID': ['F1', 'F2'], 'Lon': [100, 101], 'Lat': [10, 11]})
    updated_df = get_precipitation_data.combine_precip(df, pickle_file_path)
    # Check for all added columns
    for column in [
        'Mean_Rainfall',
        'Median_Rainfall',
        'Max_Continuous_Rainy_Days',
        'Max_Continuous_Rainfall',
        'Max_Single_Day_Rainfall',
        'Max_Rainfall_Increase'
    ]:
        assert column in updated_df.columns


#Start test:
ipytest.run()

.......

Calculating <Avg_Rain> features:   0%|          | 0/2 [00:00<?, ?it/s]

Processing <Avg_Rain> column:   0%|          | 0/2 [00:00<?, ?it/s]

.

Calculating <Mean_Rainfall> features:   0%|          | 0/2 [00:00<?, ?it/s]

...

Processing <Max_Single_Day_Rainfall> column:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating <Max_Rainfall_Increase> features:   0%|          | 0/2 [00:00<?, ?it/s]

Processing <Max_Rainfall_Increase> column:   0%|          | 0/2 [00:00<?, ?it/s]

.                                                                               [100%]
14 passed in 1.72s


<ExitCode.OK: 0>